# PyOR also called Python On Resonance
## Author: Vineeth Francis Thalakottoor
## Email: vineeth.thalakottoor@ens.psl.eu or vineethfrancis.physics@gmail.com
## Example: Zeeman basis and Singlet Triplet basis

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/Source'

# Add source path
import sys
sys.path.append(SourcePath)
import time
%matplotlib ipympl

# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_DensityMatrix import DensityMatrix
from PyOR_QuantumObject import QunObj
from PyOR_HardPulse import HardPulse
from PyOR_Basis import Basis
from PyOR_Evolution import Evolutions
from PyOR_Plotting import Plotting
import PyOR_SignalProcessing as Spro
from PyOR_Commutators import Commutators
from PyOR_QuantumLibrary import QuantumLibrary
from PyOR_Relaxation import RelaxationProcess

In [2]:
# Define the spin system
Spin_list = {"A" : "H1", "B" : "H1"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Set parameters

In [3]:

# Master Equation
QS.PropagationSpace = "Liouville"
QS.MasterEquation = "Redfield"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = 9.4

# Rotating Frame Frequency
QS.OmegaRF[0] = -QS.A.gamma*QS.B0
QS.OmegaRF[1] = -QS.B.gamma*QS.B0

# Offset Frequency in rotating frame (Hz)
QS.Offset[0] = 10.0
QS.Offset[1] = 20.0

# Define J coupling between Spins (Jlist[i][j], j > i)
QS.Jlist[0][1] = 1.0

# Define paris of spins coupled by dipolar interaction
QS.DipolePairs = [(0,1)]

# Define initial and final Spin Temperature
QS.Ispintemp[0] = 300.0
QS.Ispintemp[1] = 300.0
QS.Fspintemp[0] = 300.0
QS.Fspintemp[1] = 300.0

# Relaxation Process
QS.Rprocess = "Auto-correlated Dipolar Homonuclear"
QS.RelaxParDipole_tau = 10.0e-12
QS.RelaxParDipole_bIS = [30.0e3]

### Generate Hamiltonians

In [4]:
# generate Larmor Frequencies
QS.print_Larmor = True
Ham = Hamiltonian(QS)
COMM = Commutators()
Hz = Ham.Zeeman_RotFrame()

# J coupling Hamiltonian
Hj = Ham.Jcoupling()

# Generating the commutation superoperator
QS.RowColOrder = 'C'
QLib = QuantumLibrary(QS)

Hz_L = QLib.CommutationSuperoperator(Hz+Hj)

Larmor Frequency in MHz:  [-400.22802765 -400.22803765]


In [26]:
Hz.Inverse2PI().matrix

Matrix([
[-14.9999997343941,               0,                0,                0],
[                0, 4.9999999114647,                0,                0],
[                0,               0, -4.9999999114647,                0],
[                0,               0,                0, 14.9999997343941]])

In [25]:
Hj.Inverse2PI().matrix

Matrix([
[0.250000006956884,                  0,                  0,                 0],
[                0, -0.250000006956884,  0.500000013913768,                 0],
[                0,  0.500000013913768, -0.250000006956884,                 0],
[                0,                  0,                  0, 0.250000006956884]])

## Zeeman Basis Kets

In [5]:
QS.ZeemanBasis_Ket()

['|1/2,1/2>|1/2,1/2>',
 '|1/2,1/2>|1/2,-1/2>',
 '|1/2,-1/2>|1/2,1/2>',
 '|1/2,-1/2>|1/2,-1/2>']

## Zeeman Basis Bras

In [6]:
QS.ZeemanBasis_Bra()

['<1/2,1/2|<1/2,1/2|',
 '<1/2,1/2|<1/2,-1/2|',
 '<1/2,-1/2|<1/2,1/2|',
 '<1/2,-1/2|<1/2,-1/2|']

# Zeeman Basis states

In [7]:
BS = Basis(QS)
Zstates, DicZ = BS.Zeeman_Basis()

In [8]:
print(DicZ)

['|1/2,1/2>|1/2,1/2>', '|1/2,1/2>|1/2,-1/2>', '|1/2,-1/2>|1/2,1/2>', '|1/2,-1/2>|1/2,-1/2>']


In [9]:
Zstates[0].matrix

Matrix([
[1.0],
[  0],
[  0],
[  0]])

In [10]:
Zstates[1].matrix

Matrix([
[  0],
[1.0],
[  0],
[  0]])

In [11]:
Zstates[2].matrix

Matrix([
[  0],
[  0],
[1.0],
[  0]])

In [12]:
Zstates[3].matrix

Matrix([
[  0],
[  0],
[  0],
[1.0]])

## Singlet Triplet Basis states

In [13]:
STstates, DicST = BS.SingletTriplet_Basis()

In [14]:
print(DicST)

['Tm ', 'T0 ', 'Tp ', 'S0 ']


In [15]:
STstates[0].matrix

Matrix([
[1.0],
[  0],
[  0],
[  0]])

In [16]:
STstates[1].matrix

Matrix([
[                0],
[0.707106781186547],
[0.707106781186547],
[                0]])

In [17]:
STstates[2].matrix

Matrix([
[  0],
[  0],
[  0],
[1.0]])

In [18]:
STstates[3].matrix

Matrix([
[                 0],
[ 0.707106781186547],
[-0.707106781186547],
[                 0]])

## Transformation Between Zeeman state and Singlet-Triplet state

In [19]:
U_Z_ST = BS.BasisChange_TransformationMatrix(Zstates,STstates)

In [20]:
U_Z_ST.matrix

Matrix([
[1.0,                 0,   0,                  0],
[  0, 0.707106781186547,   0,  0.707106781186547],
[  0, 0.707106781186547,   0, -0.707106781186547],
[  0,                 0, 1.0,                  0]])

## Transformation of Zeeman and J coupling Hamiltonian into Singlet-Triplet basis

In [21]:
Hj_ST = BS.BasisChange_Operator(Hj,U_Z_ST)
Hz_ST = BS.BasisChange_Operator(Hz,U_Z_ST)

In [22]:
Hj_ST.Inverse2PI().matrix

Matrix([
[0.250000006956884,                 0,                 0,                  0],
[                0, 0.250000006956884,                 0,                  0],
[                0,                 0, 0.250000006956884,                  0],
[                0,                 0,                 0, -0.750000020870651]])

In [23]:
Hz_ST.Inverse2PI().matrix

Matrix([
[-14.9999997343941,               0,                0,               0],
[                0,               0,                0, 4.9999999114647],
[                0,               0, 14.9999997343941,               0],
[                0, 4.9999999114647,                0,               0]])